In [76]:
import pandas as pd
from fastai import *
from fastai.text.all import *
articleDetails = pd.read_pickle('headlines.pkl')

In [43]:
articleDetails.head()

,Link,Headline,Post Time
0,https://order-order.com/2022/06/14/snp-mp-patrick-grady-recommended-for-two-day-suspension-by-independent-expert-panel-over-sexual-misconduct/,SNP MP Patrick Grady Recommended for Two Day Suspension by Independent Expert Panel For Sexual Misconduct,2022-06-14 11:12:00
1,https://order-order.com/2022/06/14/sir-keirs-many-meaningless-slogans/,Sir Keir’s Many Meaningless Slogans,2022-06-14 10:24:00
2,https://order-order.com/2022/06/14/protocol-problem-produces-perils-of-pronunciation/,Protocol Problem Produces Perils of Pronunciation,2022-06-14 09:54:00
3,https://order-order.com/2022/06/14/inflation-outpacing-wages-by-biggest-margin-in-a-decade/,Inflation Outpacing Wages by Biggest Margin in a Decade,2022-06-14 08:59:00
4,https://order-order.com/2022/06/13/one-year-at-gb-news/,One Year at GB News,2022-06-13 17:24:00


In [73]:
from bs4 import BeautifulSoup
from datetime import datetime
import requests

# Function to go back and fill in headlines, and then re-export it as pkl!
indexes_where_bad_headline = articleDetails[articleDetails['Headline'] == ""].index

for i in range(0, len(indexes_where_bad_headline)):
    link = articleDetails['Link'][indexes_where_bad_headline[i]]

    page = requests.get(link)
    soup = BeautifulSoup(page.content, "html.parser")

    title = soup.find("v-card-title", {"class": "red accent-4 white--text d-block"}).text.lstrip().rstrip()
    post_time = soup.find("span", {"class": "posted-on blue-grey--text text--darken-4"}).text
    articleDetails['Headline'][indexes_where_bad_headline[i]] = title
    articleDetails['Post Time'][indexes_where_bad_headline[i]] = datetime.strptime(post_time, '%B %d %Y @ %H:%M')
    
articleDetails.to_pickle('headlines.pkl')

In [78]:
# https://order-order.com/2013/11/13/154334/
# https://order-order.com/2007/10/16/libdem-leadership-betting/

articleDetails[articleDetails['Link'] == ""]

,Link,Headline,Post Time


In [81]:
# Fix NaT in Post Time column
articleDetails[articleDetails['Post Time'] == np.datetime64('NaT')]

,Link,Headline,Post Time


In [82]:
print("We have " + str(len(articleDetails['Headline'])) + " headlines in our Corpus of data")

We have 37781 headlines in our Corpus of data


In [20]:
# Average length of headline
sum_lens = 0
for i in range(0, len(articleDetails['Headline'])):
    sum_lens += len(articleDetails['Headline'][i].split())
    
average_length = (sum_lens / len(articleDetails['Headline']))
print(average_length)

6.851953828232554


In [39]:
# Most Popular first words
def find_in_list_of_tuple(list, target):
    return_value = False
    for j in range(0, len(list)):
        if list[j][0] == target:
            return_value = True
    return return_value

def find_index_in_list_of_tuple(list, target):
    return_value = False
    for j in range(0, len(list)):
        if list[j][0] == target:
            return_value = j
    return return_value


def generateLeadingStringArray(array, num_words):
    local_arr = []
    for i in range(0, len(array)):
        first_word_of_string = ' '.join(array[i].split()[:num_words])
        if find_in_list_of_tuple(local_arr, first_word_of_string) == True:
            # Word has already been added to local_arr
            local_arr[find_index_in_list_of_tuple(local_arr, first_word_of_string)][1] += 1
        else:
            local_arr.append([first_word_of_string, 1])

    local_arr = sorted(local_arr, key=lambda tup: tup[1], reverse=True)
    
    return local_arr

In [40]:
#most_popular_first_word = generateLeadingStringArray(articleDetails['Headline'], 1)
most_popular_two_words = generateLeadingStringArray(articleDetails['Headline'], 2)
#most_popular_three_words = generateLeadingStringArray(articleDetails['Headline'], 3)

#print(most_popular_first_word[:5])
print(most_popular_two_words[:10])
#print(most_popular_three_words[:5])














































































































[['Friday Caption', 638], ['Rich’s Monday', 423], ['Saturday Seven', 413], ['Rich &', 284], ['Saturday 7-Up', 209], ['+ +', 171], ['PMQs: Who’s', 155], ['Caption Contest', 130], ['Who Is', 127], ['', 109]]


In [23]:
dls = TextDataLoaders.from_df(articleDetails, path="./", text_col='Headline', valid_pct=0.3, is_lm=True)

In [24]:
model = language_model_learner(dls, AWD_LSTM)

In [25]:
model.fit_one_cycle(5)

epoch,train_loss,valid_loss,time
0,5.522987,4.897814,29:58
1,4.861944,4.165838,29:54
2,4.354949,3.849818,30:02
3,4.102753,3.750346,29:54
4,4.025673,3.737153,29:57


In [29]:
model.export()

In [30]:
model = load_learner('export.pkl')

In [36]:
def name_of_obj(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]

def getGeneration(leading_string_arr, number_of_words=10, min_length=1, max_length=3, number_of_generations=3):
    model_output = ""
    model_output = model_output + "----------------\n"
    model_output = model_output + "Using the array: " + name_of_obj(leading_string_arr, globals())[0] + "\n"
    for i in range(0, number_of_words):
        print("'" + leading_string_arr[i][0] + "'")
        if leading_string_arr[i][0][-1] == ":":
            model_output = model_output + leading_string_arr[i][0] + "\n"
        else:
            model_output = model_output + leading_string_arr[i][0] + ":\n"
        for j in range(min_length, max_length):
            num_words = int(average_length*(j+1))
            model_output = model_output + "\t" + "With length of " + str(num_words) + ":\n"
            for k in range(0, number_of_generations):
                with model.no_bar(), model.no_logging():
                    model_output = model_output + "\t\t" + model.predict(leading_string_arr[i][0], n_words=num_words) + "\n"
            
    print(model_output)

In [37]:
#getGeneration(most_popular_first_word)
getGeneration(most_popular_two_words)
#getGeneration(most_popular_three_words)

'Friday Caption'


'Rich’s Monday'


'Saturday Seven'


'Rich &'


'Saturday 7-Up'


...

'PMQs: Who’s'


'Caption Contest'


'Who Is'


''


IndexError: string index out of range